<a href="https://colab.research.google.com/github/Baekhyeonjeong/movie-review-/blob/main/%EC%98%81%ED%99%94_%EC%8B%B1%EC%8A%A4%ED%8A%B8%EB%A6%AC%ED%8A%B8_%EB%A6%AC%EB%B7%B0_%ED%98%95%ED%83%9C%EC%86%8C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [171]:
# konlpy 설치하기
!pip install konlpy

In [172]:
# konlpy를 실행하기
import konlpy

In [173]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [174]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [175]:
## 문서 ID로 실제 파일 불러오기
# 싱스트릿.csv
## https://drive.google.com/file/d/1ARXch3FZJGy98obfG3Pzb9tDDtAJ4aOR/view?usp=sharing


rawdata_downloaded = drive.CreateFile({'id': '1ARXch3FZJGy98obfG3Pzb9tDDtAJ4aOR'})
rawdata_downloaded.GetContentFile('싱스트릿.csv')

In [176]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('싱스트릿.csv', sep=",")

In [177]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,hull(miss****),코너의 상상씬에서 부모님이 함께 춤을 추고 형이 머리 손질하고 와서 라피나 전남친 ...,9,3570,61,2016-05-19 23:02:00,관람객
1,1,수(suli****),"지금 가지 않으면, 절대 못가니까.",10,2509,49,2016-05-19 02:46:00,0
2,2,백복치(bsj4****),형이 하드캐리&진주인공 이네요!!!ㅠㅠㅠㅠ,10,2254,38,2016-05-19 10:55:00,0
3,3,홍용진(jyki****),비긴어게인과 원스랑은 또 다른 감동...곡성 시빌워도 좋지만 이런 영화가 우리 삶을...,9,1964,63,2016-05-19 00:02:00,0
4,4,박지혜(myid****),토끼 키우는 남자애 너무 귀엽지 않나요? 정말,9,1426,21,2016-05-25 12:08:00,관람객
...,...,...,...,...,...,...,...,...
4595,4595,2yez****,나만 재미없나 . 나만 재미없나 .,1,2,9,2019-02-04 22:28:00,0
4596,4596,Layla(sdh4****),이게무슨 9점짜리 영화;;,1,3,10,2017-11-11 01:07:00,0
4597,4597,호수(hrns****),full of disgusting idiots,1,4,11,2016-07-18 02:42:00,0
4598,4598,원먀(suk8****),아 내시간 ㅡㅡ,1,0,8,2019-10-01 00:27:00,0


In [178]:
원본데이터NAN삭제 = 원본데이터.dropna(axis=0)
## 바로: NAN값(빈값)을 삭제하기 위한 프로세스

In [179]:
원본데이터NAN삭제

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,hull(miss****),코너의 상상씬에서 부모님이 함께 춤을 추고 형이 머리 손질하고 와서 라피나 전남친 ...,9,3570,61,2016-05-19 23:02:00,관람객
1,1,수(suli****),"지금 가지 않으면, 절대 못가니까.",10,2509,49,2016-05-19 02:46:00,0
2,2,백복치(bsj4****),형이 하드캐리&진주인공 이네요!!!ㅠㅠㅠㅠ,10,2254,38,2016-05-19 10:55:00,0
3,3,홍용진(jyki****),비긴어게인과 원스랑은 또 다른 감동...곡성 시빌워도 좋지만 이런 영화가 우리 삶을...,9,1964,63,2016-05-19 00:02:00,0
4,4,박지혜(myid****),토끼 키우는 남자애 너무 귀엽지 않나요? 정말,9,1426,21,2016-05-25 12:08:00,관람객
...,...,...,...,...,...,...,...,...
4595,4595,2yez****,나만 재미없나 . 나만 재미없나 .,1,2,9,2019-02-04 22:28:00,0
4596,4596,Layla(sdh4****),이게무슨 9점짜리 영화;;,1,3,10,2017-11-11 01:07:00,0
4597,4597,호수(hrns****),full of disgusting idiots,1,4,11,2016-07-18 02:42:00,0
4598,4598,원먀(suk8****),아 내시간 ㅡㅡ,1,0,8,2019-10-01 00:27:00,0


# 형태소 분석

In [180]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [181]:
# "원본데이터NAN삭제"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터NAN삭제.iterrows(): # 원본데이터NAN삭제의 컬럼(행)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 대상 텍스트의 열은 2번입니다. python은 0부터 숫자를 세게 됩니다!!!
    분류 = row[3] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 평점 정보는 3번째 열에 있습니다.
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [182]:
# "형태소" 변수의 저장 내용 확인
형태소

[('중', 'Noun'),
 ('2', 'Number'),
 ('병', 'Noun'),
 ('의', 'Josa'),
 ('행진', 'Noun'),
 ('들', 'Suffix'),
 ('...', 'Punctuation'),
 ('현실', 'Noun'),
 ('물정', 'Noun'),
 ('모르는', 'Verb'),
 ('사람', 'Noun'),
 ('들', 'Suffix'),
 ('이', 'Josa'),
 ('좋아할', 'Adjective'),
 ('만', 'Noun'),
 ('한', 'Josa'),
 ('영화', 'Noun'),
 ('.', 'Punctuation'),
 ('노래', 'Noun'),
 ('몇개', 'Noun'),
 ('는', 'Josa'),
 ('좋고', 'Adjective'),
 ('라피', 'Noun'),
 ('나가', 'Verb'),
 ('너무', 'Adverb'),
 ('매력', 'Noun'),
 ('적', 'Suffix'),
 ('이어서', 'Josa'),
 ('6', 'Number'),
 ('점', 'Noun'),
 ('.', 'Punctuation'),
 ('그거', 'Noun'),
 ('빼고는', 'Verb'),
 ('쇼', 'Noun'),
 ('미', 'Adjective'),
 ('더', 'Noun'),
 ('머니가', 'Adjective'),
 ('떠오른다', 'Verb'),
 ('허세', 'Noun'),
 ('중', 'Noun'),
 ('2', 'Number'),
 ('병', 'Noun'),
 ('장애', 'Noun'),
 ('남', 'Noun'),
 ('들', 'Suffix'),
 ('그', 'Noun'),
 ('자체', 'Noun'),
 ('..', 'Punctuation'),
 ('게이', 'Noun'),
 ('들', 'Suffix'),
 ('의', 'Josa'),
 ('행진', 'Noun'),
 ('..', 'Punctuation'),
 ('남', 'Noun'),
 ('배우', 'Noun'),
 ('들', 'Suf

In [183]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,중,Noun,1,1
1,2,Number,1,1
2,병,Noun,1,1
3,의,Josa,1,1
4,행진,Noun,1,1
5,들,Suffix,1,1
6,...,Punctuation,1,1
7,현실,Noun,1,1
8,물정,Noun,1,1
9,모르는,Verb,1,1


In [184]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[       0            1  type  count
 0     코너         Noun     9      1
 1      의         Josa     9      1
 2    상상씬         Noun     9      1
 3     에서         Josa     9      1
 4    부모님         Noun     9      1
 5      이         Josa     9      1
 6     함께       Adverb     9      1
 7      춤         Noun     9      1
 8      을         Josa     9      1
 9     추고         Verb     9      1
 10     형         Noun     9      1
 11     이         Josa     9      1
 12    머리         Noun     9      1
 13    손질         Noun     9      1
 14    하고         Josa     9      1
 15    와서         Verb     9      1
 16    라피         Noun     9      1
 17     나         Josa     9      1
 18   전남친         Noun     9      1
 19     무         Noun     9      1
 20  찔러주는         Verb     9      1
 21    장면         Noun     9      1
 22     은         Josa     9      1
 23    조금         Noun     9      1
 24  뭉클했음    Adjective     9      1
 25     .  Punctuation     9      1
 26    누구         Noun     9

In [185]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [186]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,코너,Noun,9,1
1,의,Josa,9,1
2,상상씬,Noun,9,1
3,에서,Josa,9,1
4,부모님,Noun,9,1
...,...,...,...,...
54,배우,Noun,1,1
55,들,Suffix,1,1
56,의,Josa,1,1
57,반,Noun,1,1


In [187]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [188]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,코너,Noun,9,1
1,의,Josa,9,1
2,상상씬,Noun,9,1
3,에서,Josa,9,1
4,부모님,Noun,9,1
...,...,...,...,...
54,배우,Noun,1,1
55,들,Suffix,1,1
56,의,Josa,1,1
57,반,Noun,1,1


In [189]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [190]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소      품사           분류
!        Punctuation  1      2
                      6      3
                      7     13
                      8     33
                      9     81
                            ..
힘들었을     Adjective    10     1
힘들었지만    Adjective    10     1
힝목봄봇     Noun         9      1
！        Foreign      10     1
）Ｏ────♪  Foreign      10     1
Name: 카운트, Length: 16303, dtype: int64

In [191]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [192]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소     품사          분류     
!       Punctuation 1     2
                    6     3
                    7    13
                    8    33
                    9    81
...                     ...
힘들었을    Adjective   10    1
힘들었지만   Adjective   10    1
힝목봄봇    Noun        9     1
！       Foreign     10    1
）Ｏ────♪ Foreign     10    1

[16303 rows x 1 columns]

In [193]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')